

## 0. 라이브러리 설치



In [ ]:
!pip install pycaret==2.3.10

In [ ]:
!pip install jinja2==3.1.2

     |████████████████████████████████| 133 kB 4.1 MB/s 
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install xgboost==1.6.0

     |████████████████████████████████| 193.7 MB 39 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


## 1. 데이터 불러오기
전처리된 데이터 csv파일로 저장해 구글 드라이브에 올려놓고 가져와 사용하는 방식
- index, occyp_type, FLAG_MOBIL 삭제
- gender, car, reality -> 0과 1로 변경
- DAYS_BIRTH -> age 컬럼 생성 후 삭제
- DAYS_EMPLOYED -> (-1)곱하고, 음수는 0으로 변경
- begin_month -> (-1) 곱해줌
- income_total -> 로그스케일 적용

In [ ]:
from google.colab import drive

drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
import io
import pandas as pd

In [ ]:
file='/gdrive/MyDrive/Semi_3/data/pycaret_data.csv'
df = pd.read_csv(file)
df.head(3)

,Unnamed: 0,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,credit,age
0,0,0,0,0,0,12.218500,Commercial associate,Higher education,Married,Municipal apartment,4709,0,0,0,2.0,6.0,1.0,38
1,1,0,0,1,1,12.419170,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1540,0,0,1,3.0,5.0,1.0,31
2,2,1,1,1,0,13.017005,Working,Higher education,Married,House / apartment,4434,0,1,0,2.0,22.0,2.0,52


In [ ]:
del df['Unnamed: 0']

print(df.shape)
df.head(2)

(26457, 17)


,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,credit,age
0,0,0,0,0,12.21850,Commercial associate,Higher education,Married,Municipal apartment,4709,0,0,0,2.0,6.0,1.0,38
1,0,0,1,1,12.41917,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1540,0,0,1,3.0,5.0,1.0,31


## 2. pycaret 적용

In [ ]:
from pycaret.classification import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
model = setup(data= df,
              target='credit',
              numeric_features=['DAYS_EMPLOYED','child_num'], # categoric으로 분류돼서 직접 지정
              train_size=0.7,
              session_id=42)

,Description,Value
0,session_id,42
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 17)"
5,Missing Values,False
6,Numeric Features,6
7,Categorical Features,10
8,Ordinal Features,False
9,High Cardinality Features,False


## 2-1. 분류 모델 비교

In [ ]:
# 평가 지표 추가
from sklearn.metrics import log_loss

add_metric('logloss','LogLoss',log_loss,greater_is_better=False,target='pred_proba')

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7f4cdc7578c0>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [ ]:
# svm과 ridge 모델은 predict_proba를 지원하지 않아 제외
top_3_models = compare_models(sort='logloss', n_select=3, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7037,0.7089,0.4540,0.6851,0.6448,0.2770,0.3343,0.7650,7.149
lightgbm,Light Gradient Boosting Machine,0.7010,0.7017,0.4289,0.6994,0.6262,0.2428,0.3249,0.7658,0.969
gbc,Gradient Boosting Classifier,0.6924,0.6545,0.4087,0.6559,0.6075,0.2074,0.2966,0.7944,5.914
lda,Linear Discriminant Analysis,0.6449,0.6110,0.3373,0.5365,0.5116,0.0120,0.0497,0.8589,0.131
lr,Logistic Regression,0.6437,0.6133,0.3335,0.4300,0.5042,0.0005,0.0043,0.8593,1.780
dummy,Dummy Classifier,0.6436,0.5000,0.3333,0.4142,0.5040,0.0000,0.0000,0.8792,0.032
rf,Random Forest Classifier,0.6983,0.7414,0.5312,0.6752,0.6799,0.3597,0.3673,0.9290,2.217
nb,Naive Bayes,0.6276,0.5815,0.3561,0.5211,0.5433,0.0623,0.0821,1.0287,0.034
ada,Ada Boost Classifier,0.6922,0.6283,0.4039,0.6215,0.6028,0.1980,0.2987,1.0809,0.604
et,Extra Trees Classifier,0.6718,0.7042,0.5120,0.6489,0.6553,0.3107,0.3158,2.3833,2.186


In [ ]:
# xgboost모델 예측 결과값
predict_model(top_3_models[0])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Extreme Gradient Boosting,0.6973,0.7181,0.4498,0.6714,0.6356,0.2732,0.3312,0.7641


,child_num,income_total,DAYS_EMPLOYED,family_size,begin_month,age,gender_0,car_0,reality_0,income_type_Commercial associate,...,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,work_phone_1,phone_0,email_0,credit,Label,Score
0,0.0,12.660331,545.0,1.0,11.0,28.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2,0.5602
1,1.0,12.283038,1782.0,2.0,7.0,28.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2,0.6983
2,0.0,12.382129,7095.0,2.0,53.0,41.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2,0.7272
3,0.0,11.630717,0.0,2.0,14.0,66.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2,0.6766
4,1.0,12.419169,1931.0,3.0,16.0,35.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2,0.6886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7933,0.0,11.813038,0.0,1.0,-0.0,63.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0.5688
7934,0.0,12.506181,1674.0,2.0,29.0,26.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1,0.6741
7935,0.0,11.813038,1259.0,2.0,24.0,28.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2,0.4854
7936,0.0,11.669938,0.0,2.0,39.0,63.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2,0.6927


## 2-2. 분류 모델 튜닝

In [ ]:
tuned_top3 = [tune_model(i, optimize='logloss') for i in top_3_models]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.6992,0.6694,0.4185,0.7034,0.6178,0.2283,0.3206,0.7815
1,0.6987,0.6849,0.4199,0.7085,0.6190,0.2314,0.3168,0.7753
2,0.7003,0.6800,0.4232,0.7067,0.6217,0.2364,0.3238,0.7765
3,0.6879,0.6496,0.4040,0.6678,0.6013,0.1947,0.2818,0.7969
4,0.6928,0.6865,0.4134,0.6588,0.6116,0.2126,0.2972,0.7839
5,0.6928,0.6947,0.4134,0.6675,0.6117,0.2172,0.2960,0.7800
6,0.7003,0.6946,0.4202,0.6745,0.6200,0.2336,0.3234,0.7752
7,0.6874,0.6865,0.4036,0.6316,0.6018,0.1967,0.2777,0.7879
8,0.6955,0.6930,0.4126,0.6741,0.6120,0.2190,0.3063,0.7766


In [ ]:
top_3_models

[XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
               missing=nan, monotone_constraints='()', n_estimators=100,
               n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
               predictor='auto', random_state=42, reg_alpha=0, ...),
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_job

In [ ]:
tuned_top3

[XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=2,
               missing=nan, monotone_constraints='()', n_estimators=170,
               n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
               predictor='auto', random_state=42, reg_alpha=10, ...),
 LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
                importance_type='split', learning_rate=0.2, max_depth=-1,
                min_child_samples=6, min_child_wei

## 2-3. 분류 모델 블렌딩

In [ ]:
blend_top_3 = blend_models(estimator_list=tuned_top3, fold=5, optimize='logloss')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.7011,0.6931,0.4230,0.7133,0.6222,0.2381,0.3252,0.7645
1,0.6963,0.6845,0.4209,0.6803,0.6190,0.2289,0.3082,0.7761
2,0.6952,0.7153,0.4161,0.6956,0.6143,0.2209,0.3059,0.7649
3,0.6998,0.7048,0.4219,0.6924,0.6206,0.2346,0.3222,0.7647
4,0.6970,0.7064,0.4180,0.6898,0.6170,0.2277,0.3102,0.7619
Mean,0.6979,0.7008,0.4200,0.6943,0.6186,0.2300,0.3143,0.7664
Std,0.0022,0.0108,0.0025,0.0108,0.0028,0.0059,0.0078,0.0050


In [ ]:
# 블렌딩 모델로 예측
pred = predict_model(blend_top_3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6966,0.7155,0.4274,0.6915,0.6184,0.2436,0.3294,0.7645


## 2.4 마무리

In [ ]:
# finalize_model : 전체 데이터로 마지막 학습을 진행함
final_blended = finalize_model(blend_top_3)

In [ ]:
# 최종 모델로 예측
predict_model(final_blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.7108,0.8711,0.4474,0.7436,0.6385,0.2828,0.375,0.6607


,child_num,income_total,DAYS_EMPLOYED,family_size,begin_month,age,gender_0,car_0,reality_0,income_type_Commercial associate,...,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,work_phone_1,phone_0,email_0,credit,Label,Score
0,0.0,12.660331,545.0,1.0,11.0,28.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2,0.6602
1,1.0,12.283038,1782.0,2.0,7.0,28.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2,0.6178
2,0.0,12.382129,7095.0,2.0,53.0,41.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2,0.6983
3,0.0,11.630717,0.0,2.0,14.0,66.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2,0.6604
4,1.0,12.419169,1931.0,3.0,16.0,35.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2,0.6625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7933,0.0,11.813038,0.0,1.0,-0.0,63.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0.5403
7934,0.0,12.506181,1674.0,2.0,29.0,26.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1,0.5450
7935,0.0,11.813038,1259.0,2.0,24.0,28.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2,0.5702
7936,0.0,11.669938,0.0,2.0,39.0,63.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2,0.7419


## 2.5 모델 저장

In [ ]:
save_model(final_blended, '/gdrive/MyDrive/Semi_3/final_blended_model_1')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['DAYS_EMPLOYED',
                                                           'child_num'],
                                       target='credit', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_n...
                                                                           min_impurity_decrease=0.0002,
                                                                           min_impurity_split=None,
                                           